In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, LeaveOneOut
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.metrics import  make_scorer, accuracy_score, f1_score, confusion_matrix, roc_auc_score, silhouette_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics.cluster import normalized_mutual_info_score
import pickle
import joblib

In [2]:
!where python

c:\Users\polyx\Desktop\Pattern-Recognition-Gr7\venv\Scripts\python.exe
C:\Users\polyx\AppData\Local\Programs\Python\Python310\python.exe
C:\Users\polyx\AppData\Local\Microsoft\WindowsApps\python.exe


In [17]:
import sklearn
print(np.__version__)

2.1.2


In [9]:
with open('KFold_3_grid_search.pkl', 'rb') as file:
    grid_search_results = pickle.load(file)

print(grid_search_results.cv_results_)

{'mean_fit_time': array([80.67851122, 78.94051059, 78.6826663 , 85.13051112, 84.543564  ,
       79.36713211, 78.49887594, 81.47484032, 79.40421788, 79.77389654,
       81.6503226 , 84.22803617, 77.87267057, 86.61179662, 84.278241  ,
       89.67706545, 78.38694263, 79.9967773 , 82.55887707, 79.81250763,
       78.1470507 , 79.73844123, 80.43918578, 82.83642721, 81.08245301,
       78.89450097, 79.4717919 , 80.11236183, 79.44231399, 81.53421084,
       80.87033351, 80.27751533, 77.02105443, 76.71711   , 93.84230622,
       79.89837718, 77.54749346, 77.2642614 , 77.44071007, 77.53262933,
       77.93907364, 78.6999975 , 79.20396233, 78.89447308, 78.2551113 ,
       77.29721117, 77.72843409, 77.79722047, 75.64867862, 80.64607279,
       74.62920713, 83.91037552, 77.34965213, 77.4633464 , 77.55452879,
       76.92251643, 77.45570628, 79.40206734, 78.42390037, 77.88254523,
       78.53954442, 78.97717222, 77.19995968, 78.49303047, 82.50482281,
       75.92954675, 83.14541141, 77.56646299, 

In [10]:
def create_ranked_lists(grid_search_results):

    cv_results = grid_search_results.cv_results_
    
    df = pd.DataFrame({
        'mean_fit_time': cv_results['mean_fit_time'],
        'mean_score_time': cv_results['mean_score_time'],
        'mean_test_f1': cv_results['mean_test_f1'],
        'params': cv_results['params']  
    })

    # Extract F1 scores
    f1_scores = cv_results.get('mean_test_f1', None)
    
    if f1_scores is not None:
        # Count how many have a perfect F1 score 
        perfect_f1_count = sum(1 for score in f1_scores if score == 1.0)
        print(f"Number of models with a perfect F1 score: {perfect_f1_count}")


    # Filter out models that dont have perfect f1 score
    df_perfect_f1 = df[df['mean_test_f1'] == 1.0]



    # Rank models based on mean_score_time
    df_ranked_by_score_time = df_perfect_f1.sort_values(by='mean_score_time', ascending=True).reset_index(drop=True)

    # Rank models based on mean_fit_time
    df_ranked_by_fit_time = df_perfect_f1.sort_values(by='mean_fit_time', ascending=True).reset_index(drop=True)

    # Rank models based on the combined time
    df_perfect_f1['combined_time'] = df_perfect_f1['mean_score_time'] + df_perfect_f1['mean_fit_time']
    df_ranked_by_combined_time = df_perfect_f1.sort_values(by='combined_time', ascending=True).reset_index(drop=True)


    return df_ranked_by_score_time, df_ranked_by_fit_time, df_ranked_by_combined_time

In [11]:
ranked_by_score_time, ranked_by_fit_time, ranked_by_combined_time = create_ranked_lists(grid_search_results)

# Print Ranked by Mean Score Time
print("Ranked by Mean Score Time:")
print(ranked_by_score_time[['mean_score_time', 'mean_test_f1', 'params']])

# Print Ranked by Mean Fit Time
print("\nRanked by Mean Fit Time:")
print(ranked_by_fit_time[['mean_fit_time', 'mean_test_f1', 'params']])

# Print Ranked by Combined Time (mean_score_time + mean_fit_time)
print("\nRanked by Combined Time:")
print(ranked_by_combined_time[['combined_time', 'mean_test_f1', 'params']])

Number of models with a perfect F1 score: 10
Ranked by Mean Score Time:
   mean_score_time  mean_test_f1  \
0         0.085107           1.0   
1         0.087116           1.0   
2         0.097032           1.0   
3         0.097084           1.0   
4         0.131665           1.0   
5         0.157523           1.0   
6         0.177575           1.0   
7         0.201358           1.0   
8         0.207283           1.0   
9         0.503205           1.0   

                                              params  
0  {'classifier': SVC(probability=True, random_st...  
1  {'classifier': SVC(probability=True, random_st...  
2  {'classifier': SVC(probability=True, random_st...  
3  {'classifier': SVC(probability=True, random_st...  
4  {'classifier': SVC(probability=True, random_st...  
5  {'classifier': SVC(probability=True, random_st...  
6  {'classifier': KNeighborsClassifier(), 'classi...  
7  {'classifier': KNeighborsClassifier(), 'classi...  
8  {'classifier': KNeighborsClassifi

C:\Users\polyx\AppData\Local\Temp\ipykernel_35648\3384443238.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_perfect_f1['combined_time'] = df_perfect_f1['mean_score_time'] + df_perfect_f1['mean_fit_time']


In [12]:
ranked_by_combined_time[['params'][0]][0]

{'classifier': KNeighborsClassifier(),
 'classifier__n_neighbors': 5,
 'classifier__weights': 'distance',
 'feature_selection': 'passthrough'}